**Корректность проверена на Python 3.6:**
+ pandas 0.23.4
+ numpy 1.15.4
+ sklearn 0.20.2

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [2]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [3]:
iris = datasets.load_iris()

In [4]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [5]:
classifier = linear_model.SGDClassifier(random_state = 0, tol=1e-3)

### Генерация сетки

In [6]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [7]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : np.arange(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [8]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [9]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [10]:
%%time
grid_cv.fit(train_data, train_labels)

Wall time: 6.98 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'squared_loss'],
                         'max_iter': array([5, 6, 7, 8, 9]),
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [11]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, max_iter=9, penalty='l1',
              random_state=0)

In [12]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


In [13]:
grid_cv.cv_results_

{'mean_fit_time': array([0.00279934, 0.00239921, 0.00249927, 0.00360112, 0.00279889,
        0.00249901, 0.00259945, 0.00239966, 0.00279925, 0.00249918,
        0.00249932, 0.00249946, 0.00249927, 0.00249903, 0.00259905,
        0.00279865, 0.00269902, 0.002595  , 0.00279913, 0.00269971,
        0.00239906, 0.0023988 , 0.00259936, 0.00251408, 0.00249913,
        0.00249875, 0.00249887, 0.0024991 , 0.00299883, 0.00299897,
        0.00259888, 0.0023994 , 0.0025032 , 0.00249927, 0.00259931,
        0.00249898, 0.00259922, 0.00259902, 0.0026201 , 0.00249884,
        0.00359383, 0.0032002 , 0.00269899, 0.00239968, 0.00249915,
        0.00249944, 0.00249834, 0.00230236, 0.0025991 , 0.00249925,
        0.00239933, 0.0023994 , 0.0023994 , 0.00259895, 0.00269806,
        0.00249937, 0.00249684, 0.00249953, 0.00259852, 0.00249908,
        0.00239937, 0.00239906, 0.00239906, 0.00229883, 0.00249929,
        0.00239892, 0.0024992 , 0.00259912, 0.00250194, 0.00239882,
        0.00249934, 0.00249915,

#### Randomized grid search

In [15]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0)

In [16]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

Wall time: 726 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   estimator=SGDClassifier(random_state=0), n_iter=20,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                                        'loss': ['hinge', 'log',
                                                 'squared_hinge',
                                                 'squared_loss'],
                                        'max_iter': array([5, 6, 7, 8, 9]),
                                        'penalty': ['l1', 'l2']},
                   random_state=0, scoring='accuracy')

In [17]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}


In [20]:
#!/usr/bin/python

from datetime import datetime


def control_sum(str):
    return sum(map(ord, list(str)))


def check(reply):
    def _is_number(str):
        try:
            int(str)
            return True
        except ValueError:
            return False

    if "Control sum:" not in reply:
        return False
    parts = reply.split("Control sum:")
    received_current_time = parts[0].strip()
    received_control_sum = parts[1].strip()
    if not _is_number(received_control_sum):
        return False
    else:
        received_control_sum = int(received_control_sum)
    expected_control_sum = control_sum(received_current_time)
    return expected_control_sum == received_control_sum


current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)
print("Control sum: " + str(control_sum(current_time)))

print(check(current_time + '\n' + "Control sum: " + str(control_sum(current_time))))

2021-02-19 14:40:52
Control sum: 943
True
